In [1]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
meta = pd.read_csv('movies_metadata.csv', low_memory=False)

In [3]:
meta.head(1)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0


In [4]:
meta.iloc[0]

adult                                                                False
belongs_to_collection    {'id': 10194, 'name': 'Toy Story Collection', ...
budget                                                            30000000
genres                   [{'id': 16, 'name': 'Animation'}, {'id': 35, '...
homepage                              http://toystory.disney.com/toy-story
id                                                                     862
imdb_id                                                          tt0114709
original_language                                                       en
original_title                                                   Toy Story
overview                 Led by Woody, Andy's toys live happily in his ...
popularity                                                       21.946943
poster_path                               /rhIRbceoE9lR4veEXuwCC2wARtG.jpg
production_companies        [{'name': 'Pixar Animation Studios', 'id': 3}]
production_countries     

In [5]:
vote_data = meta[['original_title', 'vote_average', 'vote_count']]

In [6]:
vote_data[vote_data.vote_count>=14075]

,original_title,vote_average,vote_count
15480,Inception,8.1,14075.0


In [7]:
vote_data['vote_count'].mean()

109.89733831940167

In [8]:
vote_data['vote_average'].mean()

5.618207215133889

In [9]:
def weighted_rating(row):
    v = row['vote_count']
    m = 1500
    r = row['vote_average']
    c = 5.6
    wr = ((v/(v+m))*r)+((m/(v+m))*c)
    return wr

In [10]:
((12269/(12269+1500))*8.3)+((1500/(12269+1500))*5.6)

8.005860992083667

In [11]:
vote_data['score']=vote_data.apply(weighted_rating, axis=1)

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [12]:
vote_data.sort_values(by='score',ascending=False).head(10)

,original_title,vote_average,vote_count,score
314,The Shawshank Redemption,8.5,8358.0,8.058734
12481,The Dark Knight,8.3,12269.0,8.005861
2843,Fight Club,8.3,9678.0,7.937681
834,The Godfather,8.5,6024.0,7.921850
292,Pulp Fiction,8.3,8670.0,7.901770
15480,Inception,8.1,14075.0,7.859230
22879,Interstellar,8.1,11187.0,7.804422
351,Forrest Gump,8.2,8147.0,7.795729
7000,The Lord of the Rings: The Return of the King,8.1,8226.0,7.714436
1154,The Empire Strikes Back,8.2,5998.0,7.679861


In [13]:
vote_data.sort_values(by='vote_count', ascending=False).head(10)

,original_title,vote_average,vote_count,score
15480,Inception,8.1,14075.0,7.859230
12481,The Dark Knight,8.3,12269.0,8.005861
14551,Avatar,7.2,12114.0,7.023711
17818,The Avengers,7.4,12000.0,7.200000
26564,Deadpool,7.4,11444.0,7.191409
22879,Interstellar,8.1,11187.0,7.804422
20051,Django Unchained,7.8,10297.0,7.520268
23753,Guardians of the Galaxy,7.9,10014.0,7.600365
2843,Fight Club,8.3,9678.0,7.937681
18244,The Hunger Games,6.9,9634.0,6.724861


# Content Based

### Based on Genre

In [14]:
from ast import literal_eval

> Convert string to List of Object with Literal_eval

In [15]:
#movie disortir dengan vote count diatas 300
cb_genres = meta[meta['vote_count']>300][['original_title','genres']]

In [16]:
cb_genres.reset_index(drop=True, inplace=True)

In [17]:
cb_genres['genres'][0]

"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

In [18]:
meta.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [19]:
literal_eval(meta['genres'][0])

[{'id': 16, 'name': 'Animation'},
 {'id': 35, 'name': 'Comedy'},
 {'id': 10751, 'name': 'Family'}]

In [20]:
def new_genre(x):
    genres = []
    x = literal_eval(x)
    for item in x:
        genres.append(item['name'])
    return ' '.join(genres)
    

In [21]:
cb_genres['new_genres'] = cb_genres['genres'].apply(new_genre)

In [22]:
cb_genres.head(1)

,original_title,genres,new_genres
0,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",Animation Comedy Family


> Convert to Sparsematrix with countVectorizer

In [23]:
from sklearn.feature_extraction.text import CountVectorizer

In [24]:
cb_genres.head(1)

,original_title,genres,new_genres
0,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",Animation Comedy Family


In [25]:
cv=CountVectorizer()

In [26]:
cv_genres_result=cv.fit_transform(cb_genres['new_genres'])

In [27]:
cv_df=pd.DataFrame(cv_genres_result.todense(), columns=cv.get_feature_names(), index=cb_genres['original_title'])

In [28]:
meta.genres.loc[0]

"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

In [29]:
cv_df.head()

,action,adventure,animation,comedy,crime,documentary,drama,family,fantasy,fiction,...,horror,movie,music,mystery,romance,science,thriller,tv,war,western
original_title,,,,,,,,,,,,,,,,,,,,,
Toy Story,0,0,1,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
Jumanji,0,1,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
Heat,1,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
GoldenEye,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
Balto,0,1,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
#Measure similarity
#cosine similarity

In [31]:
data_1 = 'animation comedy'
data_2 = 'animation adventure'
data_3='horror comedy'
data_4='animation'
data_5='animation comedy'

In [32]:
data=[data_1, data_2, data_3, data_4, data_5]
data=pd.DataFrame(data=data, columns=['genre'], index=['movie 1', 'movie 2', 'movie 3', 'movie 4', 'movie 5'])

In [33]:
data

,genre
movie 1,animation comedy
movie 2,animation adventure
movie 3,horror comedy
movie 4,animation
movie 5,animation comedy


In [34]:
cv_2 = CountVectorizer()
cv_res = cv_2.fit_transform(data['genre'])

In [35]:
pd.DataFrame(cv_res.todense(), columns=cv_2.get_feature_names(), index=data.index)

,adventure,animation,comedy,horror
movie 1,0,1,1,0
movie 2,1,1,0,0
movie 3,0,0,1,1
movie 4,0,1,0,0
movie 5,0,1,1,0


In [36]:
from sklearn.metrics.pairwise import cosine_similarity

In [38]:
pd.DataFrame(cos_sim, index=data.index, columns=data.index)

,movie 1,movie 2,movie 3,movie 4,movie 5
movie 1,1.000000,0.500000,0.5,0.707107,1.000000
movie 2,0.500000,1.000000,0.0,0.707107,0.500000
movie 3,0.500000,0.000000,1.0,0.000000,0.500000
movie 4,0.707107,0.707107,0.0,1.000000,0.707107
movie 5,1.000000,0.500000,0.5,0.707107,1.000000


> Simmilarity for our data

In [39]:
cos_sim_movie = cosine_similarity(cv_genres_result)

In [40]:
def get_recommendation_based_genres(title):
    index_to_search = cb_genres[cb_genres['original_title']==title].index[0]
    series_similar=pd.Series(cos_sim_movie[index_to_search])
    index_similar=series_similar.sort_values(ascending=False).head(10).index
    return cb_genres.loc[index_similar]

In [41]:
get_recommendation_based_genres('Harry Potter and the Deathly Hallows: Part 2')

,original_title,genres,new_genres
1618,City of Ember,"[{'id': 12, 'name': 'Adventure'}, {'id': 10751...",Adventure Family Fantasy
2007,Harry Potter and the Deathly Hallows: Part 2,"[{'id': 10751, 'name': 'Family'}, {'id': 14, '...",Family Fantasy Adventure
1299,Harry Potter and the Goblet of Fire,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",Adventure Fantasy Family
2901,Pete's Dragon,"[{'id': 12, 'name': 'Adventure'}, {'id': 10751...",Adventure Family Fantasy
862,Harry Potter and the Philosopher's Stone,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",Adventure Fantasy Family
165,The Wizard of Oz,"[{'id': 12, 'name': 'Adventure'}, {'id': 10751...",Adventure Family Fantasy
2666,Pan,"[{'id': 12, 'name': 'Adventure'}, {'id': 10751...",Adventure Family Fantasy
1577,The Chronicles of Narnia: Prince Caspian,"[{'id': 12, 'name': 'Adventure'}, {'id': 10751...",Adventure Family Fantasy
402,Labyrinth,"[{'id': 12, 'name': 'Adventure'}, {'id': 10751...",Adventure Family Fantasy
1303,"The Chronicles of Narnia: The Lion, the Witch ...","[{'id': 12, 'name': 'Adventure'}, {'id': 10751...",Adventure Family Fantasy


### Based on Cast and Director

In [42]:
credits=pd.read_csv('credits.csv')

In [43]:
credits.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [44]:
id_movie = pd.to_numeric(meta[meta['vote_count']>300]['id'].reset_index(drop=True))
cb_genres['id']=id_movie

In [45]:
cb_genres_cast_crew = cb_genres.merge(credits,on='id')

In [46]:
cb_genres_cast_crew.head(1)

,original_title,genres,new_genres,id,cast,crew
0,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",Animation Comedy Family,862,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."


In [47]:
cb_genres_cast_crew.shape

(3032, 6)

In [48]:
def new_cast(x):
    casts = []
    x = literal_eval(x)
    for item in x:
        name = item['name'].replace(' ','')
        casts.append(name)
    return ' '.join(casts)

In [49]:
cb_genres.head(1)

,original_title,genres,new_genres,id
0,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",Animation Comedy Family,862


In [50]:
credits.head(1)

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862


In [51]:
cb_genres_cast_crew.head(1)

,original_title,genres,new_genres,id,cast,crew
0,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",Animation Comedy Family,862,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."


In [52]:
cb_genres_cast_crew['new_cast'] = cb_genres_cast_crew['cast'].apply(new_cast)

In [53]:
cv_res_cast = CountVectorizer().fit_transform(cb_genres_cast_crew['new_cast'])

In [54]:
cos_sim_cast = cosine_similarity(cv_res_cast)

In [55]:
def get_recommendation_based_cast(title):
    index_to_search = cb_genres_cast_crew[cb_genres_cast_crew['original_title'] == title].index[0]
    series_similar = pd.Series(cos_sim_cast[index_to_search])
    index_similar = series_similar.sort_values(ascending=False).head(10).index
    return cb_genres_cast_crew.loc[index_similar][['original_title','new_cast']]

In [56]:
get_recommendation_based_cast('Jumanji')

,original_title,new_cast
1,Jumanji,RobinWilliams JonathanHyde KirstenDunst Bradle...
448,Halloween: H20,JamieLeeCurtis JoshHartnett AdamArkin Michelle...
634,The Green Mile,TomHanks MichaelClarkeDuncan DavidMorse Bonnie...
1693,Moon,SamRockwell KevinSpacey DominiqueMcElligott Ro...
138,Jack,RobinWilliams DianeLane BrianKerwin JenniferLo...
317,Anaconda,JenniferLopez IceCube JonVoight EricStoltz Owe...
81,Ri¢hie Ri¢h,MacaulayCulkin JohnLarroquette EdwardHerrmann ...
29,The Birdcage,RobinWilliams GeneHackman NathanLane DianneWie...
188,Aladdin and the King of Thieves,ScottWeinger LindaLarkin RobinWilliams JohnRhy...
1154,Wimbledon,KirstenDunst PaulBettany SamNeill JamesMcAvoy ...


In [57]:
cos_sim_cast_genres =(cos_sim_movie*0.6)+(cos_sim_cast*0.4)

In [58]:
pd.DataFrame(cos_sim_cast_genres, columns=cb_genres_cast_crew.original_title, index=cb_genres_cast_crew.original_title)

original_title,Toy Story,Jumanji,Heat,GoldenEye,Balto,Casino,Sense and Sensibility,Four Rooms,Ace Ventura: When Nature Calls,Get Shorty,...,Valerian and the City of a Thousand Planets,What Happened to Monday,Atomic Blonde,Dunkirk,Transformers: The Last Knight,Tre uomini e una gamba,The Dark Tower,The Emoji Movie,Shot Caller,Girls Trip
original_title,,,,,,,,,,,,,,,,,,,,,
Toy Story,1.000000,0.200000,0.000000,0.000000,0.400000,0.016182,0.000000,0.244949,0.200000,0.200000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.346410,0.000000,0.600000,0.000000,0.346410
Jumanji,0.200000,1.000000,0.000000,0.200000,0.400000,0.000000,0.000000,0.000000,0.200000,0.000000,...,0.173205,0.000000,0.000000,0.000000,0.154919,0.000000,0.141421,0.200000,0.000000,0.000000
Heat,0.000000,0.000000,1.000000,0.346410,0.000000,0.438520,0.212132,0.212132,0.173205,0.358262,...,0.150000,0.173205,0.424264,0.402492,0.268328,0.000000,0.129251,0.000000,0.519615,0.000000
GoldenEye,0.000000,0.200000,0.346410,1.000000,0.200000,0.000000,0.000000,0.000000,0.200000,0.200000,...,0.346410,0.200000,0.489898,0.309839,0.464758,0.000000,0.141421,0.000000,0.200000,0.000000
Balto,0.400000,0.400000,0.000000,0.200000,1.000000,0.000000,0.000000,0.000000,0.200000,0.000000,...,0.182510,0.000000,0.000000,0.000000,0.164976,0.000000,0.000000,0.400000,0.000000,0.000000
Casino,0.016182,0.000000,0.438520,0.000000,0.000000,1.000000,0.300000,0.300000,0.244949,0.244949,...,0.000000,0.000000,0.000000,0.189737,0.000000,0.000000,0.000000,0.000000,0.497191,0.000000
Sense and Sensibility,0.000000,0.000000,0.212132,0.000000,0.000000,0.300000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.189737,0.000000,0.000000,0.000000,0.000000,0.244949,0.000000
Four Rooms,0.244949,0.000000,0.212132,0.000000,0.000000,0.300000,0.000000,1.000000,0.489898,0.489898,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.424264,0.000000,0.244949,0.244949,0.424264
Ace Ventura: When Nature Calls,0.200000,0.200000,0.173205,0.200000,0.200000,0.244949,0.000000,0.489898,1.000000,0.400000,...,0.173205,0.000000,0.000000,0.000000,0.154919,0.346410,0.000000,0.200000,0.200000,0.346410


In [59]:
def get_recommendation_based_genre_cast(title):
    df_cos_sin_genre_cast=pd.DataFrame(cos_sim_cast_genres, columns=cb_genres_cast_crew.original_title, index=cb_genres_cast_crew.original_title)
    movie_similar=df_cos_sin_genre_cast[title]
    return movie_similar.sort_values(ascending=False).head(10)

In [60]:
get_recommendation_based_genre_cast('Titanic')

original_title
Titanic                 1.000000
Kill Your Darlings      0.600000
아가씨                     0.600000
Fifty Shades of Grey    0.600000
Victoria                0.600000
Cruel Intentions        0.600000
Notorious               0.600000
The Beach               0.527617
Match Point             0.525447
The Illusionist         0.519615
Name: Titanic, dtype: float64

### Based on Crew

In [61]:
#def new_cast(x):
  #  casts = []
   # x = literal_eval(x)
   # for item in x:
   #     name = item['name'].replace(' ','')
     #   casts.append(name)
  #  return ' '.join(casts)

In [62]:
#def new_director(x):
   # for item in literal_eval(x):
     #   if(item['job']=='Director'):
      #      return item['name'].replace(' ','')

In [63]:
#cb_genres_cast_crew['new_crew'] = cb_genres_cast_crew['crew'].apply(new_director)

In [64]:
#cv_res_crew = CountVectorizer().fit_transform(cb_genres_cast_crew['new_crew'])

In [65]:
#cos_sim_crew = cosine_similarity(cv_res_crew)

In [66]:
#cos_sim_dir_genre_cast = (cos_sim_movie*0.5)+(cos_sim_cast*0.4)+(cos_sim_crew*0.1)

In [67]:
#def get_recommendation_based_dir_genre_cast(title):
    #df_cos_sin_dir_genre_cast=pd.DataFrame(cos_sim_dir_genre_cast, columns=cb_genres_cast_crew.original_title, index=cb_genres_cast_crew.original_title)
    #movie_similar=df_cos_sin_dir_genre_cast[title]
    #return movie_similar.sort_values(ascending=False).head(10)

In [68]:
#get_recommendation_based_dir_genre_cast('Ice Age: The Meltdown')

# Collaborative Filtering

In [69]:
# User based -----> selera user
# item based ----->

### Item Based:

In [70]:
ratings = pd.read_csv('ratings_small.csv')

In [71]:
ratings.shape

(100004, 4)

In [72]:
ratings.head(1)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144


In [73]:
index_to_drop=meta[meta.id.str.contains('-')].index
meta.drop(index_to_drop, inplace=True)

In [74]:
#convert to numeric
meta.id=pd.to_numeric(meta.id)

In [75]:

meta.rename({'id':'movieId'}, axis=1, inplace=True)

In [76]:
user_movie_ratings = meta.merge(ratings,on='movieId')[['userId','original_title','rating']]

In [77]:
user_movie_ratings.original_title.describe()

count                                  44994
unique                                  2798
top       Terminator 3: Rise of the Machines
freq                                     324
Name: original_title, dtype: object

In [78]:
user_movie_ratings_pivot=user_movie_ratings.pivot_table(index='original_title', columns='userId', values='rating')

In [79]:
user_movie_ratings_pivot.loc['!Women Art Revolution'].isnull().sum()

669

In [80]:
user_movie_ratings_pivot.dropna(thresh=10,inplace=True)

In [81]:
user_movie_ratings_pivot.fillna(0, inplace=True)

In [82]:
similarity_movie=cosine_similarity(user_movie_ratings_pivot)

In [83]:
df_item_similarity=pd.DataFrame(similarity_movie, index=user_movie_ratings_pivot.index, columns=user_movie_ratings_pivot.index)

In [84]:
users = [['titanic',5],['A Perfect Murder',2],['2 Days in Paris',4]]

In [85]:
df_item_similarity['Titanic'].sort_values(ascending=False)

original_title
Titanic                                  1.000000
Big Fish                                 0.655192
Psycho                                   0.651088
Reservoir Dogs                           0.616171
A Nightmare on Elm Street                0.607010
Monsoon Wedding                          0.606741
Rain Man                                 0.579327
Romeo + Juliet                           0.572078
Wag the Dog                              0.571416
Sissi                                    0.561254
Batman Returns                           0.546437
The Hours                                0.542332
Солярис                                  0.527823
The Conversation                         0.524053
Night on Earth                           0.523796
Back to the Future Part II               0.516263
Dave Chappelle's Block Party             0.514190
Aliens vs Predator: Requiem              0.506406
Bang Boom Bang - Ein todsicheres Ding    0.504994
Terminator 3: Rise of the Machines 

In [86]:
ser_1=(df_item_similarity['Titanic']*(5-2.5)).sort_values(ascending=False)
ser_2=(df_item_similarity['A Perfect Murder']*(2-2.5)).sort_values(ascending=False)
ser_3=(df_item_similarity['2 Days in Paris']*(4-2.5)).sort_values(ascending=False)

In [87]:
(ser_1+ser_2+ser_3).sort_values(ascending=False).head(10)

original_title
Titanic                      2.730555
2 Days in Paris              1.962323
Psycho                       1.774745
A Nightmare on Elm Street    1.724984
Monsoon Wedding              1.680280
Big Fish                     1.679542
Reservoir Dogs               1.603702
Sissi                        1.597969
Rain Man                     1.572642
Wag the Dog                  1.533183
dtype: float64

In [88]:
def get_cf_item_based(movies):
    serr=[]
    for item in movies:
        ser_similar=df_item_similarity[item[0]]*(item[1]-2.5)
        serr.append(ser_similar)
    similar_total=sum(serr)
    return similar_total.sort_values(ascending=False).head(10)

In [89]:
get_cf_item_based([['Titanic',2],['A Perfect Murder',5]])

original_title
A Perfect Murder           2.407151
Cold Mountain              1.110334
Аэлита                     1.078367
Prime                      1.034842
Yella                      1.002543
Frankenstein               0.981409
Lonely Hearts              0.970602
Dr. Jekyll and Mr. Hyde    0.969958
Payback                    0.964822
8 femmes                   0.939310
dtype: float64

In [90]:
df_item_similarity['Titanic']

original_title
...Più forte ragazzi!            0.461279
10 Items or Less                 0.101762
12 + 1                           0.144806
2 Days in Paris                  0.215603
20,000 Leagues Under the Sea     0.327575
2001: A Space Odyssey            0.314148
24 Hour Party People             0.215596
25th Hour                        0.057205
28 Days Later                    0.215763
28 Weeks Later                   0.210820
300                              0.262352
4 luni, 3 săptămîni și 2 zile    0.170489
42nd Street                      0.127345
48 Hrs.                          0.491408
5 Card Stud                      0.319688
5x2                              0.082737
7 vírgenes                       0.287232
8 Mile                           0.088778
8 femmes                         0.223741
88 Minutes                       0.250039
A Bridge Too Far                 0.179566
A Brief History of Time          0.248441
A Clockwork Orange               0.456351
A Close Shave      

# User Based

In [91]:
ratings.userId.describe()

count    100004.000000
mean        347.011310
std         195.163838
min           1.000000
25%         182.000000
50%         367.000000
75%         520.000000
max         671.000000
Name: userId, dtype: float64

In [92]:
user_rating_pivot=ratings.pivot_table(index='userId',columns='movieId', values='rating')

In [93]:
user_rating_pivot=user_rating_pivot.dropna(thresh=30).fillna(0)

In [94]:
#Measure similarity each users

In [95]:
user_similarity_score=cosine_similarity(user_rating_pivot)

In [96]:
df_similarity=pd.DataFrame(user_similarity_score, index=user_rating_pivot.index, columns=user_rating_pivot.index)

In [97]:
user_rating_pivot.loc[2].sort_values(ascending=False).head()

movieId
265    5.0
266    5.0
592    5.0
590    5.0
589    5.0
Name: 2, dtype: float64

In [98]:
#menambahkan user baru ke dataframe, dengan contoh user baru ratingnya sama dengan user dari dataframe(user_rating_pivot, loc[2]
new_user = [[265,5],[266,5],[592,5],[590,5],[589,5]]

In [99]:
user_rating_pivot.loc[(user_rating_pivot.index[-1]+1),[265,266,592,590,589]]=[5,5,5,5,5]

In [100]:
user_rating_pivot.fillna(0,inplace=True)

In [101]:
similarity=cosine_similarity(user_rating_pivot)

In [102]:
user_id_similar=pd.DataFrame(similarity,columns=user_rating_pivot.index, index=user_rating_pivot.index).loc[672].sort_values(ascending=False)[1:11].index

In [103]:
#untuk user baru yang sudah diberi rekomendasi, untuk collaborativenya memnggunakan mean dari 10 user yang ratingnya similar, 
#agar filmnya tidak ditonton lagi (collaborative)

In [104]:
serr = []
for id in user_id_similar:
    movie_rating=user_rating_pivot.loc[id]
    serr.append(movie_rating)

In [105]:
(sum(serr)/10).sort_values(ascending=False)

movieId
589      4.2
590      4.2
377      4.0
480      3.7
356      3.6
150      3.5
457      3.4
592      3.2
588      3.1
364      3.1
165      3.0
380      3.0
110      2.9
349      2.9
316      2.8
266      2.7
527      2.6
296      2.6
593      2.5
318      2.4
153      2.4
47       2.4
344      2.3
595      2.3
440      2.2
292      2.1
500      2.1
434      2.1
357      2.1
265      2.1
        ... 
31700    0.0
31696    0.0
31694    0.0
31689    0.0
31682    0.0
32031    0.0
31660    0.0
31658    0.0
31549    0.0
31547    0.0
31524    0.0
31522    0.0
31747    0.0
31770    0.0
31804    0.0
31878    0.0
31903    0.0
31921    0.0
31923    0.0
31930    0.0
31952    0.0
31956    0.0
31963    0.0
31973    0.0
32017    0.0
32019    0.0
32022    0.0
32025    0.0
32029    0.0
6247     0.0
Length: 9066, dtype: float64

In [106]:
def get_recomendation_user_based(ratings):
    movie_index=np.array(ratings)[:,0]
    movie_rating=np.array(ratings)[:,1]
    user_rating_pivot.loc[(user_rating_pivot.index[-1]+1), movie_index]=movie_rating
    user_rating_pivot.fillna(0, inplace=True)
    similarity=cosine_similarity(user_rating_pivot)
    user_id_similar=pd.DataFrame(similarity, columns=user_rating_pivot.index, index=user_rating_pivot.index).loc[user_rating_pivot.index[-1]].sort_values(ascending=False)[1:11]
    
    serr=[]
    for id in user_id_similar:
        movie_rating=user_rating_pivot.loc[id]
        serr.append(movie_rating)
    return (sum(serr)/10).sort_values(ascending=False)

In [107]:
#([[356,5],[1,5],[110,3]])